In [199]:
# !pip install altair --upgrade

In [200]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [201]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [202]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [203]:
%%capture pwd
!pwd

In [204]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('will-our-energy-use-change-after-the-pandemic-and-cop26', 300, 500)

# Fig 1

In [205]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx",
    sheet_name="Figure 1",
    usecols="A:D",
    skiprows=1,
)
df.columns = ["year", "gdp", "energy", "co2"]

In [206]:
f = "fig1_gdp_energy_emissions"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,gdp,energy,co2
0,2019,0.0,0.0,0.0
1,2020,-3.5,-4.0,-5.8
2,2021,2.2,0.5,-1.2


In [207]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=2,
            format=".0f",
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "gdp:Q",
        sort=[],
        axis=alt.Axis(
            gridOpacity=0.4,
            title="Indexed to 2019=0%",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-6, 5]),
    )
)
label1 = line1.mark_text(
    text="GDP", color=colors["eco-turquiose"], align="left", dx=5
).transform_filter("datum.year==2021")
line2 = base.mark_line(color=colors["eco-light-blue"]).encode(
    y=alt.Y("energy:Q", sort=[])
)
label2 = line2.mark_text(
    text="Energy demand", color=colors["eco-light-blue"], align="left", dx=5
).transform_filter("datum.year==2021")
line3 = base.mark_line(color=colors["eco-gray"]).encode(y=alt.Y("co2:Q", sort=[]))
label3 = line3.mark_text(
    text="CO₂ emissions", color=colors["eco-gray"], align="left", dx=5
).transform_filter("datum.year==2021")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": 2019, "y": 0}, {"x": 2021, "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"], opacity=0.5)
    .encode(x=alt.X("x:Q", sort=[]), y="y:Q")
)
layer1 = (
    (
        (line1 + axis1 + line2 + label2 + label1 + label3 + line3).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [208]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx", sheet_name="Figure 2"
)

In [209]:
f = "fig2_energy_by_source"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Year,Fuel,EJ
0,1965,Renewables,0.179852
1,1966,Renewables,0.198060
2,1967,Renewables,0.199866
3,1968,Renewables,0.221200
4,1969,Renewables,0.232580


In [210]:
base = (
    alt.Chart(f2)
    .encode(
        x=alt.X(
            "Year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                format=".0f",
                orient="bottom",
                labelAngle=0,
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.Year+' | '+datum.Fuel+ ': '+round(datum.EJ*10)/10 +' EJ'"
    )
    .transform_calculate(
        order="indexof(['Coal','Oil','Gas','Renewables','Hydro','Nuclear'], datum.Fuel)"
    )
)
area = base.mark_area(fillOpacity=0.7, line={},).encode(
    y=alt.Y(
        "EJ:Q",
        stack=True,
        axis=alt.Axis(
            title="EJ",
            domain=False,
            gridOpacity=0.4,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            format=".0f",
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "Fuel:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-blue"],
                colors["eco-yellow"],
                colors["eco-light-blue"],
                colors["eco-dot"],
                colors["eco-gray"],
                colors["eco-green"],
            ]
        ),
    ),
    order=alt.Order("order:Q", sort="descending"),
)
labels = (
    area.mark_text(align="left", dy=10, dx=5)
    .encode(text="Fuel:N")
    .transform_filter("datum.Year==2020")
)
layer1 = (
    ((area + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [211]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx",
    sheet_name="Figure 3",
    usecols="A:B",
)

In [212]:
f = "fig3_emissions"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Year,World CO2 Emissions (GTCO2)
0,2000,23.663544
1,2001,23.995243
2,2002,24.539478
3,2003,25.755703
4,2004,27.076611


In [213]:
base = (
    alt.Chart(f3)
    .encode(
        x=alt.X(
            "Year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=5,
                format=".0f",
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_calculate(
        order="indexof(['Coal','Oil','Gas','Renewables','Hydro','Nuclear'], datum.Fuel)"
    )
)
area = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        line={"color": colors["eco-mid-blue"]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-mid-blue"], offset=0.7),
                alt.GradientStop(color="white", offset=0),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        y=alt.Y(
            "World CO2 Emissions (GTCO2):Q",
            stack=True,
            axis=alt.Axis(
                title="GtCO₂",
                domain=False,
                gridOpacity=0.4,
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=20,
                format=".0f",
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleFontWeight="normal",
                titleAlign="left",
                tickCount=6,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.Year+' | '+round(datum[\"World CO2 Emissions (GTCO2)\"]*10)/10.0+ ' GtCO₂'"
    )
    .transform_filter("datum.Year<=2020")
)
line = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        line={"color": colors["eco-light-blue"], "strokeDash": [5, 5]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-light-blue"], offset=0.7),
                alt.GradientStop(color="white", offset=0),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(y="World CO2 Emissions (GTCO2):Q", tooltip="tooltip:N")
    .transform_calculate(
        tooltip="datum.Year+' | '+round(datum[\"World CO2 Emissions (GTCO2)\"]*10)/10.0+ ' GtCO₂ (estimated)'"
    )
    .transform_filter("datum.Year>=2020")
)
label1 = area.mark_text(
    align="left",
    dy=-10,
    dx=5,
    text="CO₂ emissions in 2020",
    color=colors["eco-mid-blue"],
).transform_filter("datum.Year==2020")
label2 = line.mark_text(
    align="left",
    dy=10,
    dx=5,
    text="Global CO₂ emissions",
    color=colors["eco-light-blue"],
).transform_filter("datum.Year==2030")
line2 = (
    alt.Chart(pd.DataFrame([{"x": 2020, "y": 32}, {"x": 2030, "y": 32}]))
    .mark_line(strokeDash=[5, 5], strokeWidth=1)
    .encode(x="x:Q", y="y:Q")
)
point = (
    alt.Chart(pd.DataFrame([{"x": 2030, "y": 28.5}]))
    .mark_point(size=50, fill=colors["eco-green"], color=colors["eco-turquiose"])
    .encode(x="x:Q", y="y:Q")
)
label3 = point.mark_text(
    align="left",
    dy=18,
    dx=-40,
    text="IEA WEO 2021 SDS scenario",
    color=colors["eco-green"],
)
layer1 = (
    (
        (area + label1 + label2 + line + line2 + point + label3).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 4

In [214]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx", sheet_name="Figure 4"
)
df = df.set_index("Shares").stack().reset_index()
df.columns = ["Year", "Fuel", "Share"]
df = df.reset_index()

In [215]:
f = "fig4_energy_shares"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,index,Year,Fuel,Share
0,0,2010,Oil,0.341385
1,1,2010,Gas,0.225133
2,2,2010,Coal,0.299195
3,3,2010,Renewables,0.019053
4,4,2010,Nuclear,0.051420


In [216]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "Year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                format=".0f",
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_calculate(
        order="indexof(['Coal','Oil','Gas','Renewables','Hydro','Nuclear'], datum.Fuel)"
    )
    .transform_filter('datum.Year!="WEO 2020 SDS - 2030"')
)
area = base.mark_area(fillOpacity=0.7, line={},).encode(
    y=alt.Y(
        "Share:Q",
        stack=True,
        axis=alt.Axis(
            title="",
            domain=False,
            gridOpacity=0.4,
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            format=".0%",
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "Fuel:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-blue"],
                colors["eco-yellow"],
                colors["eco-light-blue"],
                colors["eco-dot"],
                colors["eco-gray"],
                colors["eco-green"],
            ]
        ),
    ),
    order=alt.Order("order:Q", sort="descending"),
)
labels = (
    area.mark_text(align="left", dy=10, dx=5)
    .encode(text="Fuel:N")
    .transform_filter("datum.Year==2030")
)
layer1 = (
    ((area + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## Fig 4 recalculated

In [217]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx", sheet_name="Figure 4"
)
df = df.set_index("Shares")
df["Total"] = df.T.sum()
for c in df.columns:
    df[c] /= df["Total"]
df = df.drop("Total", axis=1).stack().reset_index()
df.columns = ["Year", "Fuel", "Share"]
df = df.reset_index()

In [218]:
f = "fig4_energy_shares_revised"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,index,Year,Fuel,Share
0,0,2010,Oil,0.341385
1,1,2010,Gas,0.225133
2,2,2010,Coal,0.299195
3,3,2010,Renewables,0.019053
4,4,2010,Nuclear,0.051420


In [219]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "Year:Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                titleAlign="center",
                titleAnchor="middle",
                title="",
                titleY=-15,
                titleX=207,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                format=".0f",
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_calculate(
        order="indexof(['Coal','Oil','Gas','Renewables','Hydro','Nuclear'], datum.Fuel)"
    )
    .transform_filter('datum.Year!="WEO 2020 SDS - 2030"')
)
area = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "Share:Q",
            stack=False,
            axis=alt.Axis(
                title="",
                gridOpacity=0.4,
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        color=alt.Color(
            "Fuel:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-blue"],
                    colors["eco-yellow"],
                    colors["eco-light-blue"],
                    colors["eco-dot"],
                    colors["eco-gray"],
                    colors["eco-green"],
                ]
            ),
        ),
        order=alt.Order("order:Q", sort="descending"),
        tooltip="tooltip:N",
    )
    .transform_calculate(
        tooltip="datum.Year+' | '+datum.Fuel+ ': '+round(datum.Share*1000)/10 +'%'"
    )
    .transform_filter("datum.Year<=2020")
)
area2 = (
    base.mark_line(strokeDash=[5, 5])
    .encode(y="Share:Q", color="Fuel:N", tooltip="tooltip:N")
    .transform_calculate(
        tooltip="datum.Year+' | '+datum.Fuel+ ': '+round(datum.Share*1000)/10 +'% (estimated)'"
    )
    .transform_filter("datum.Year>=2020")
)
labels = (
    area2.mark_text(align="left", dy=0, dx=5)
    .encode(text="Fuel:N")
    .transform_filter("datum.Year==2030")
)
rule = (
    alt.Chart(pd.DataFrame([{"x": 2020}]))
    .mark_rule(opacity=0.1, color=colors["eco-gray"], strokeWidth=1)
    .encode(x="x:Q")
)
tt = area.mark_line(strokeWidth=30, opacity=0.01)
tt2 = area2.mark_line(strokeWidth=30, opacity=0.01)
layer1 = (
    ((area + labels + area2 + rule + tt + tt2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 5

In [220]:
df = pd.read_excel(
    "raw/EE_&_RF_-_Data_for_Figures_and_Tables.xlsx", sheet_name="Figure 5"
)

In [221]:
f = "fig5_sector"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Sector,share
0,Buildings,8.305896
1,Food,0.708421
2,Industry,15.988183
3,Transport,56.466750
4,Other,18.530749


In [222]:
base = alt.Chart(f5).encode(theta=alt.Theta("share:Q", stack=True))

pie = base.mark_arc(innerRadius=100, thetaOffset=1.02).encode(
    color=alt.Color("Sector:N", legend=None), order="share:Q",tooltip="tooltip:N",
    ).transform_calculate(
        tooltip="datum.Sector+': '+round(datum.share*10)/10 +'%'"
    
)
text = base.mark_text(radius=70, thetaOffset=1.02).encode(
    text="Sector:N", color=alt.Color("Sector:N"), order="share:Q"
)
text2 = (
    base.mark_text(radius=125, thetaOffset=1.02, color="white")
    .encode(text=alt.Text("t:Q", format=".0%"), order="share:Q")
    .transform_calculate(t="datum.share/100")
)
layer1 = (
    ((text2 + pie + text2 + text).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)